<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/status_posts_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Looking at responses to just Status posts



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "status_posts"
MODEL_NAME = "status_posts"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 20
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.1MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
from google.colab import files 
uploaded = files.upload() 


Saving facebook_congress_posts.csv to facebook_congress_posts.csv


In [0]:
import io 
posts = pd.read_csv(io.BytesIO(uploaded['facebook_congress_posts.csv'])) 


In [6]:
status_posts = posts[posts['post_type']=='status']
status_posts.head()

,op_id,op_gender,post_id,post_text,post_type
8,57265377,M,8,President Trump has just nominated judge Neil Gorsuch to the Supreme Court. His strong academic ...,status
13,57265377,M,13,I'll be joining The Sean Hannity Show from the GOP policy retreat in Philadelphia at 3:30pm CT. ...,status
171,57265377,M,171,"I salute my friend, Ted Cruz on standing strong for conservatism and running a campaign founded ...",status
230,57265377,M,230,Happy to be at #CPAC in Washington D.C. We'll be broadcasting LIVE in 15 minutes! Stay tuned.,status
252,57265377,M,252,"No one fought harder for strict constructionism than Antonin Scalia. As a jurist, he was profoun...",status


In [0]:
status_post_ids = status_posts['post_id'].to_list()

In [8]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
train_df = train_df[train_df['post_id'].isin(status_post_ids)]
dev_df = dev_df[dev_df['post_id'].isin(status_post_ids)]

In [11]:
len(train_df)

1944161

In [12]:
len(dev_df)

390738

In [13]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(768673, 8)

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [16]:
train_df.op_gender.value_counts()

M    582285
W    186388
Name: op_gender, dtype: int64

In [17]:
dev_df.op_gender.value_counts()

M    304590
W     86148
Name: op_gender, dtype: int64

In [0]:
###########################
# Execute this if want to do upsampling.
###########################
# from sklearn.utils import resample

# male = train_df[train_df.op_gender=='M']
# female = train_df[train_df.op_gender=='W']

# female_upsampled = resample(female,
#                           replace=True, # sample with replacement
#                           n_samples=len(male), # match number in majority class
#                           random_state=27) # reproducible results
# upsampled_train = pd.concat([male,female_upsampled])
# print(upsampled_train.op_gender.value_counts())


In [0]:
# replace `train_df` with `upsampled_train` if using upsampling.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

# replace `train_df` with `upsampled_train` if using upsampling.
new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [22]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 00 minutes, 58 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [25]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
| [4 files][  2.1 GiB/  2.1 GiB]   99.2 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
- [5 files][  2.9 GiB/  2.9 GiB]   24.6 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [28]:
embedding_matrix.shape

(200494, 100)

In [29]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           20049400  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
# smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
# smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [33]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 768673 samples, validate on 390738 samples
Epoch 1/20
768673/768673 [==============================] - 22s 28us/sample - loss: 0.5146 - acc: 0.7757 - val_loss: 0.5085 - val_acc: 0.7794
Epoch 2/20
768673/768673 [==============================] - 21s 27us/sample - loss: 0.4900 - acc: 0.7854 - val_loss: 0.5032 - val_acc: 0.7807
Epoch 3/20
768673/768673 [==============================] - 21s 27us/sample - loss: 0.4803 - acc: 0.7890 - val_loss: 0.5126 - val_acc: 0.7776
Epoch 4/20
768673/768673 [==============================] - 21s 27us/sample - loss: 0.4733 - acc: 0.7917 - val_loss: 0.5131 - val_acc: 0.7804
Epoch 5/20
768673/768673 [==============================] - 21s 27us/sample - loss: 0.4673 - acc: 0.7935 - val_loss: 0.5133 - val_acc: 0.7772
Epoch 6/20
768673/768673 [==============================] - 22s 29us/sample - loss: 0.4617 - acc: 0.7958 - val_loss: 0.5284 - val_acc: 0.7791
Epoch 7/20
768673/768673 [==============================] - 21s 27us/sample - loss: 0.4571 - acc

In [34]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.8153
Testing Accuracy:  0.7618


In [35]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.5929831405942775


In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [40]:
wrong_preds.op_gender.value_counts()

W    76530
M    16549
Name: op_gender, dtype: int64

In [41]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.06696814745430442

In [42]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2026308,11075700,86569077,W,390295,Mitch,this is also the same congress that appears to be backing out of its automatic budget cuts. yes?,Kirsten Gillibrand,Congress_Democratic,0.814346,M
2066185,11205939,48870736,W,395026,Robert,and what would have the national guard spend the money on instead? diapers for illegal aliens b...,Betty McCollum,Congress_Democratic,0.813361,M
1916591,10965983,86569077,W,389341,Karyn,"thank you, senator. our children deserve the best education that can be provided. you are sendin...",Kirsten Gillibrand,Congress_Democratic,0.616172,M
2065204,11204958,48870736,W,394962,Margaret,gary....dont worry about temp jobs.worry about a pipeline company that carries no insurance....t...,Betty McCollum,Congress_Democratic,0.787097,M
2049928,11099320,86569077,W,390678,Valerie,you people are amazing.... your hate and resentment runs so deep... why do not you have the ball...,Kirsten Gillibrand,Congress_Democratic,0.990578,M
1246868,7373436,94431309,W,277121,Bonnie,"we are praying for you and your little one....miracles happen every day, and, i will ask our fat...",Jaime Herrera Beutler,Congress_Republican,0.809938,M
369680,2649976,13876473,W,69514,Eric,zachary horwitz,Carolyn Maloney,Congress_Democratic,0.957427,M
1914895,10964287,86569077,W,389341,Tia,thank you for listening to your constituents!,Kirsten Gillibrand,Congress_Democratic,0.940306,M
2075791,11216771,77234050,W,398737,Dean,you drunk when you listened to it?,Cathy McMorris Rodgers,Congress_Republican,0.947859,M
1924124,10973516,86569077,W,389341,Joan,thank you,Kirsten Gillibrand,Congress_Democratic,0.731837,M


In [43]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2061464,11201218,48870736,W,394447,Lisa,the interior secretary pick ryan zinke (montana) is another climate change denier who wants to s...,Betty McCollum,Congress_Democratic,0.999973,M
1245848,7372416,94431309,W,277121,Douglas,"dan and jaime, you have our prayers, and our support in whatever the future brings. doug and sha...",Jaime Herrera Beutler,Congress_Republican,0.999968,M
2131812,11272792,77234050,W,399775,Dwight,"rest in peace mr, delbert belton sir! thank you for your service to our country.",Cathy McMorris Rodgers,Congress_Republican,0.999967,M
2073750,11214730,77234050,W,398697,Michelle,"""""it has unfortunate today's protests were designed to deliberately keep her office from serving...",Cathy McMorris Rodgers,Congress_Republican,0.999966,M
1934493,10983885,86569077,W,389389,Richard,"sen gillibrand ., donald trump won and he made hilary look incompetent",Kirsten Gillibrand,Congress_Democratic,0.999963,M
2127224,11268204,77234050,W,399741,Susan,thank you for listening to us/the american people and all you are doing on our behalf. stand str...,Cathy McMorris Rodgers,Congress_Republican,0.999957,M
2136033,11277013,77234050,W,399884,Gordon,"600,000 women and children will lose food assistance by september if we do not stop the federal ...",Cathy McMorris Rodgers,Congress_Republican,0.999957,M
2073615,11214595,77234050,W,398697,Danielle,wow yeah you are doing a bang up job of protecting us from those five year olds. http://www.in...,Cathy McMorris Rodgers,Congress_Republican,0.999954,M
1932423,10981815,86569077,W,389363,Allyson,"please staunchly and thoroughly reject him, and any other bigots he puts forth. also, remember m...",Kirsten Gillibrand,Congress_Democratic,0.999952,M
1905645,10955037,86569077,W,389321,Keith,says the senator who takes millions from wall street. https://www.opensecrets.org/politicians/c...,Kirsten Gillibrand,Congress_Democratic,0.999944,M


In [44]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
27492,2145407,38794897,M,36556,Debbie,i also saw you on rachel maddow's show. i am going to ask my senators to support you if they ar...,Sherrod Brown,Congress_Democratic,0.000023,W
1525171,7943465,99469231,M,296395,Bradley,thank you mr. mccain! you are a real american hero and i thank you for your continued support to...,John McCain,Congress_Republican,0.000045,W
304758,2569293,59227815,M,63440,Kayla,thank you amy nederend thiel. our family is strong and all of us need to show that strength!!,Paul Ryan,Congress_Republican,0.000056,W
39368,2157283,38794897,M,36780,Sandy,"yes, paul you are correct women will never be able to do certain things, like spell our names in...",Sherrod Brown,Congress_Democratic,0.000103,W
1058175,5307229,46506154,M,161416,Christine,hope amanda recovers quickly. may god watch over her.,Marco Rubio,Congress_Republican,0.000104,W
475449,3750465,89414884,M,126788,Coleen,"you too, tammy.",Pete Visclosky,Congress_Democratic,0.000129,W
7250,2125165,38794897,M,36281,Carol,"I am thankful, also, sherrod, for your actions.",Sherrod Brown,Congress_Democratic,0.000131,W
1748998,10272135,32570461,M,382075,Mary,hilary needs to get rid of her sidekick huma abedin,Tim Kaine,Congress_Democratic,0.000131,W
1835635,10358772,55333293,M,383538,Cecelia,"and, what is rep. congresswoman suzanne bonamici doing?",Peter DeFazio,Congress_Democratic,0.000252,W
48154,2166069,38794897,M,36849,Shaina,"why would you ever vote for mandel?! he did not even do his job as treasurer, but sure, lets pro...",Sherrod Brown,Congress_Democratic,0.000262,W
